In [76]:
# –––– Purpose of this script ––––

# It can be useful to advertise your team's current response time in an article
# This script will find and update the response time in a given article using this formatting: 'XXh XXm'. 
# Eg. 'we are responding to tickets in 5h 12m this week'

# The script will:
# 1. Pull the content of an article from your support desk
# 2. Calculate the support team's recent average response time from a Freshdesk report
# 3. Update the article content with the new average response time




In [77]:
# –––– Preperations before this script can run –––– 

# Schedule a weekly export of ticket properties (not activities) in Freshdesk and set it to receive the file by API.
# https://support.freshdesk.com/support/solutions/articles/226461-export-ticket-properties-from-your-helpdesk

# Provide some details:

desk_name = 'your_Freshdesk_account_name'
# This is your Freshdesk URL without '.freshdesk.com' on the end. 
# eg: if your full URL is 'megashop.freshdesk.com' use 'megashop'

auth_deets = ("your_API_key", "your_Freshdesk_password") 
# Your Freshdesk API key is in your Freshdesk profile: https://support.freshdesk.com/support/solutions/articles/215517-how-to-find-your-api-key
# A password must be provided but it does not appear to be verified. I haven't confirmed with Freshdesk if this is a bug. 
# For security's sake, you can leave the password as it is or enter in some humourous nonsense for your own satisfaction. 

target_article = '12345678901'
# This is an 11 digit number, which you will see in the URL when viewing that article.
# eg: https://megashop.freshdesk.com/support/solutions/articles/12345678901

precursor = 'average response time is'
# The precursor should be the few words leading up to your response time. 
# It doesn't need to be much, and should not cross any new lines.

export_URL = ''
# This is the URL at the bottom of the Freshdesk reports page when you schedule the report and opt to receive it via the API
# It should look something like: 'https://megashop.freshdesk.com/reports/scheduled_exports/1234567890123456/download_file.json'


In [78]:
# –––– Import libraries –––– 
import requests as r
import json
import pandas as pd
import re
from datetime import date, datetime, timedelta
from io import StringIO



# –––– Prepare function to report query errors –––– 

def reporter(query):
    if query.status_code != 200:
        print("Query didn't work: ", query.status_code, query.reason, query.url)

            

# –––– Prepare desk URL for API calls –––– 

desk_URL = 'https://' + desk_name + '.freshdesk.com'




In [84]:
# –––– 1. Pull the content of target article from support desk –––– 


# Query API to get the article in its current form
art_query = r.get(desk_URL + "/api/v2/solutions/articles/" + target_article, 
                  auth = auth_deets)

# Report any query errors
reporter(art_query) 

# Convert query from json and pull description field from it
art_content = art_query.json()['description']

# Locate the current response time string
old_response_time = re.search('(' + precursor + ')(.*?)([0-9]*?h [0-9]?[0-9]?m)', art_content).group(3)

print('The existing response time is ' + old_response_time)



The existing response time is 12h 4m


In [85]:
# –––– 2. Calculate the support team's recent average response time –––– 

tix_query = r.get(export_URL, auth = auth_deets)

reporter(tix_query) 

# Extract the report's URL from the response
tix_response = tix_query.json()
tix_second_query_link = tix_response['export']['url']

# Query again for this week's report
tix_second_query = r.get(tix_second_query_link)
reporter(tix_second_query)

# Convert to dataframe 
tix = pd.read_csv(StringIO(str(tix_second_query.content,'utf-8')))

# Cast response times to timedeltas
durations = ['First Response Time (in Hrs)','Resolution Time (in Hrs)']
    
for d in durations:
    tix[d] = pd.to_timedelta(tix[d])
    
# Calculate the current response time
response_time = tix['First Response Time (in Hrs)'][tix['Agent interactions'] > 0].mean()

# Convert timedelta to nice format
hours_from_days = response_time.days*24
total_minutes = response_time.seconds/60
hours_from_minutes = int(total_minutes // 60)
remaining_minutes = int(total_minutes % 60)

total_hours = hours_from_days + int(round(hours_from_minutes,0))
total_minutes = remaining_minutes

final_response_time = str(total_hours) + 'h ' + str(total_minutes) + 'm'

print('The new response time is ' + final_response_time)




The new response time is 12h 4m


In [83]:
# –––– 3. Update the article content with the new average response time –––– 

# Prepare new_art_content for PUT query. We are updating the description field
new_art_content = re.sub(old_response_time, final_response_time, art_content)
new_art_put_content = {'description' : new_art_content}

# Put-query the article string back into the live article
new_art_putter = r.put(desk_URL + "/api/v2/solutions/articles/" + target_article,
                       json = new_art_put_content,
                       auth = auth_deets)

reporter(new_art_putter)